# Model testing

- It's just a testing ground for the model. One model is loaded and a `cmc.test_model` is applied on it.
- As a result confusion matrix, ROC, AUROC, F1, PR curves will be generated and saved to the model's folder

In [1]:
import os
from pathlib import Path
import confinement_mode_classifier as cmc
import torchvision
import torch
from tqdm import tqdm
import torch.nn as nn
import re

path = Path(os.getcwd())
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

In [2]:

data_dir_path = f'{path}/data/LH_alpha'
file_names = os.listdir(data_dir_path)


#Chose what shots will be used in notebook. Removed shots has different dimensions
batch_size = 32

shot_numbers = [re.search(r'shot_(\d+)', file_name).group(1) for file_name in file_names]
removed_shots = ['19915', '19925', '13182', '20009', '20112', 
                 '20143', '20145', '20146', '20147', '20144', '20098']
shot_numbers = [valid_shot for valid_shot in shot_numbers if valid_shot not in removed_shots]

shots_for_testing = ['18130', '16773', '16534', '19094', '18133', '17837', '18128']
shots_for_validation = ['16769', '19379', '18057', '18132', '18261', '18267', '18260']


shot_df, test_df, _, _ = cmc.load_and_split_dataframes(path, shot_numbers, shots_for_testing, 
                                                       shots_for_validation=shots_for_validation,
                                                       use_ELMS=False)


### Load the model

In [7]:
#Initiate a single camera model
pretrained_model = torchvision.models.resnet18(weights='IMAGENET1K_V1', )
# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = pretrained_model.fc.in_features
pretrained_model.fc = nn.Linear(num_ftrs, 3) #3 classes: L-mode, H-mode, ELM
pretrained_model = pretrained_model.to(device)

#Which model to test?
model_path = f'{path}/runs/24-02-22, 14-36-03 RIS1, onecycleLR, extended dset_last_fc'
#TEST one camera model
pretrained_model.load_state_dict(torch.load(f'{model_path}/model.pt'))
pretrained_model.eval()
pretrained_model.to(device)

#TEST ensembled model
# ensembled_model = cmc.TwoImagesModel(modelA=pretrained_model, modelB=pretrained_model, hidden_units=30).to(device)
# ensembled_model.load_state_dict(torch.load(f'{model_path}/model.pt'))
# ensembled_model.eval()
# ensembled_model.to(device)

test_dataloader = cmc.get_dloader(test_df, path=path, batch_size=batch_size, shuffle=False, 
                                  balance_data=False, ris_option='RIS1', num_workers=32)

### Test model on unbalanced test dset

In [13]:
from importlib import reload
reload(cmc)

<module 'confinement_mode_classifier' from '/compass/Shared/Users/bogdanov/vyzkumny_ukol/confinement_mode_classifier.py'>

In [14]:
metrics = cmc.test_model(model_path, pretrained_model, test_dataloader, max_batch=0)

Processing batches: 100%|██████████| 436/436 [00:45<00:00,  9.52it/s]


### Here the model is tested on individual shots (generates time-confidence graph)

In [9]:
shots_for_testing = list(map(int,shots_for_testing))
img_path = cmc.per_shot_test(path=model_path, shots=shots_for_testing, results_df=metrics['prediction_df'])

 14%|█▍        | 1/7 [00:00<00:00,  7.03it/s]/compass/Shared/Users/bogdanov/vyzkumny_ukol/confinement_mode_classifier.py:603: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  conf_time_fig, conf_time_ax = plt.subplots(figsize=(10,6))
100%|██████████| 7/7 [00:01<00:00,  6.58it/s]
